In [3]:
import pandas as pd
import numpy as np

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, recall_score, precision_score

In [15]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.2 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/Users/nafi/.pyenv/versions/3.8.15/bin/python3.8 -m pip install --upgrade pip' command.


In [17]:
!pip install openmp

ERROR: Could not find a version that satisfies the requirement openmp (from versions: none)
ERROR: No matching distribution found for openmp
You should consider upgrading via the '/Users/nafi/.pyenv/versions/3.8.15/bin/python3.8 -m pip install --upgrade pip' command.


In [16]:
from xgboost import XGBClassifier

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/nafi/.pyenv/versions/3.8.15/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <04125532-9495-3051-97BC-F23BE76BA2F9> /Users/nafi/.pyenv/versions/3.8.15/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (n

In [4]:
ls

XGBoost.ipynb           only_motor_2.csv        only_self_report_2.csv
new_merge_2.csv         only_non_motor_2.csv


In [5]:
raw_df = pd.read_csv('only_motor_2.csv')

In [21]:
raw_df.iloc[:, 1]

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
10009    0.0
10010    1.0
10011    0.0
10012    1.0
10013    1.0
Name: CLASS, Length: 10014, dtype: float64

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
from sklearn.model_selection import StratifiedKFold

def XGBoost_model(filepath):
    fold_accuracies = []
    fold_recalls = []
    fold_precisions = []
    fold_classification_reports = []
    fold_feature_importances = []
    
    df = pd.read_csv(filepath)
    X = df.iloc[:, 2:]  # Assuming features start from the third column
    Y = df.iloc[:, 1]   # Assuming the label is in the second column

    cv = StratifiedKFold(n_splits=7)  # Corrected parameter

    for fold, (train_index, test_index) in enumerate(cv.split(X, Y)):
        print(f"fold: {fold+1}")

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

        # Initialize the model for each fold
        model = XGBClassifier()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)

        # Scoring
        accuracy = accuracy_score(Y_test, Y_pred)
        recall = recall_score(Y_test, Y_pred, average='macro')
        precision = precision_score(Y_test, Y_pred, average='macro')
        report = classification_report(Y_test, Y_pred, output_dict=True)

        # Append metrics
        fold_accuracies.append(accuracy)
        fold_recalls.append(recall)
        fold_precisions.append(precision)
        fold_classification_reports.append(report)

        # Feature importance (specific to XGBoost, since coef_ is not applicable)
        fold_feature_importances.append(model.feature_importances_)

        print(f"Accuracy for fold {fold + 1}: {accuracy:.4f}")
        print(f"Recall for fold {fold + 1}: {recall:.4f}")
        print(f"Precision for fold {fold + 1}: {precision:.4f}")
        print("-----")

    return fold_accuracies, fold_recalls, fold_precisions, fold_classification_reports, fold_feature_importances
